# Split Data

- Time series split, since the data is not iid and have time series structure
- how to create autoregression features

- Describe briefly how you split the data and why. What preprocessors did you use and why? How
many features and data points do you have before and after preprocessing. If you have missing
values, describe the basic properties of the missing values (e.g., what fraction of the points have
missing values, which features, what fraction of the features are missing, etc).

- What can we say about the provided dataset?

# Preprocessing

- We do not fill missing values at this stage
- We do not include feature engineering at this state, but will include lagged time data.
- Categorical: imbalanced flag (?), stock_id (this will largely increase feature dimension & introduce a lot of zeros to our model)
- Check feature type of every feature

- if you have missing values in continuous features, describe the basic properties of the missing values (e.g., what fraction of the points have missing values, which features, what fraction of the features are missing, etc). do not impute or drop features/rows! you will need to apply the advanced missing value techniques we will cover during the second half of the term